# Great Circle


## Overview

This section covers great circle functions from NCL:

- [area_poly_sphere](#area-poly-sphere)
- [css2c](#css2c)
- [csc2s](#csc2s)
- [gc_onarc](#gc-onarc)
- [gc_qarea](#gc-qarea)
- [gc_tarea](#gc-tarea)
- [gc_latlon](#gc-latlon)
- [gc_inout](#gc-inout)

## Functions

### area_poly_sphere
NCL's [`area_poly_sphere`](https://www.ncl.ucar.edu/Document/Functions/Built-in/area_poly_sphere.shtml) calculates the area enclosed by an arbitrary polygon on the sphere

```{admonition} Important Note
Coordinates should be within the valid latitude/longitude range (-90° to 90° and -180° to 180°) and be in clockwise order
```

Due to the shape of the Earth, the radius varies, but can be assumed to be a unit sphere with a radius of 6370997 m (based on the Clarke 1866 Authalic Sphere{footcite}`usgs_1987` model)

#### Grab and Go

In [ ]:
from pyproj import Geod

# Points in clockise order: Boulder, Boston, Houston
latitudes = [40.0150, 42.3601, 29.5518]  # degrees
longitudes = [-105.2705, -71.0589, -95.0982]  # degrees

geod = Geod(ellps="sphere")  # radius = 6370997 m
poly_area_m, _ = geod.polygon_area_perimeter(longitudes, latitudes)
poly_area_km2 = abs(poly_area_m) * 1e-6
poly_area_km2

### css2c
NCL's [`css2c`](https://www.ncl.ucar.edu/Document/Functions/Built-in/css2c.shtml) converts spherical (latitude/longitude) coordinates to Cartesian coordinates on a unit sphere

#### Grab and Go

In [ ]:
from astropy.coordinates.representation import UnitSphericalRepresentation
from astropy import units

lat = 40.0150
lon = -105.2705

spherical_coords = UnitSphericalRepresentation(lat=lat * units.deg, lon=lon * units.deg)
cart_coords = spherical_coords.to_cartesian()
print(f"X = {cart_coords.x.value}")
print(f"Y = {cart_coords.y.value}")
print(f"Z = {cart_coords.z.value}")

### csc2s
NCL's [`csc2s`](https://www.ncl.ucar.edu/Document/Functions/Built-in/csc2s.shtml) converts Cartesian coordinates to spherical (latitude/longitude) coordinates on a unit sphere

#### Grab and Go

In [ ]:
from astropy.coordinates.representation import (
    CartesianRepresentation,
    SphericalRepresentation,
)
import numpy as np

x = -0.20171369272651396
y = -0.7388354627678497
z = 0.6429881376224998

cart_coords = CartesianRepresentation(x=x, y=y, z=z)
spherical_coords = cart_coords.represent_as(SphericalRepresentation)

# convert latitude/longitude from radians to degrees
lat_deg = np.rad2deg(spherical_coords.lat.value)
lon_deg = (
    np.rad2deg(spherical_coords.lon.value) + 180
) % 360 - 180  # keep longitude between -180 to 180

print(f"Latitude = {lat_deg}")
print(f"Longitude = {lon_deg}")

### gc_onarc
NCL's [`gc_onarc`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_onarc.shtml) determines if a point on the globe lies on a specified great circle arc as long as the angle between the two points along the great circle arc is not exactly 180 degrees (diametrically opposite, or antipodal).

#### Grab and Go

In [ ]:
import numpy as np


# Convert latitude and longitude points to Cartesian Points (see: css2c)
def latlon_to_cart(lat, lon):
    from astropy.coordinates.representation import UnitSphericalRepresentation
    from astropy import units

    spherical_coords = UnitSphericalRepresentation(
        lat=lat * units.deg, lon=lon * units.deg
    )
    cart_coords = spherical_coords.to_cartesian()
    return np.array([cart_coords.x, cart_coords.y, cart_coords.z])


pt_within = latlon_to_cart(40.0150, -105.2705)  # Boulder
vertex_a = latlon_to_cart(50.0150, -105.2705)  # Point exactly 10 degrees above Boulder
vertex_b = latlon_to_cart(30.0150, -105.2705)  # Point exactly 10 degrees below Boulder

# Determine if point lies along great circle arc
from uxarray.grid.arcs import point_within_gca

print(
    f"Boulder lies within the great circle arc = {point_within_gca(pt_within, vertex_a, vertex_b)}"
)

### gc_qarea

NCL's [`gc_qarea`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_qarea.shtml) calculates the area of a (four-sided) quadrilateral patch on the unit sphere

```{admonition} Important Note
Coordinates should be within the valid latitude/longitude range (-90° to 90° and -180° to 180°) and be in clockwise or counter-clockwise order
```

#### Grab and Go

##### Area on the unit sphere

NCL's [`gc_qarea`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_qarea.shtml) function finds the area of a quadrilateral patch on the unit sphere, a sphere with radius of 1

In [ ]:
# Unit Sphere radius = 1
from pyproj import Geod

# Points in clockise order
latitudes = [90.0, 0.0, -90.0, 0.0]
longitudes = [0.0, -90.0, 0.0, 90.0]

# Adjust the radius of the spherical datum to describe the unit sphere
radius = 1

geod = Geod(a=radius)
poly_area, _ = geod.polygon_area_perimeter(longitudes, latitudes)
poly_area = abs(poly_area)
poly_area

##### Area on a spherical datum with Earth radius

`pyproj` includes [additional ellipsoid options](https://proj.org/en/stable/usage/ellipsoids.html#built-in-ellipsoid-definitions), but the ellipsoid `sphere` treats the Earth as a sphere with an equal radius of 6370997 meters

In [ ]:
# Normal Sphere: radius = 6370997 m
from pyproj import Geod

# Points in clockise order: Roughly Four Corners of Colorado
latitudes = [41.00488, 41.00203, 37.00540, 37.00051]  # degrees
longitudes = [-109.05001, -102.05348, -103.04633, -109.04720]  # degrees

geod = Geod(ellps="sphere")  # radius = 6370997 m
poly_area_m, _ = geod.polygon_area_perimeter(longitudes, latitudes)
poly_area_km2 = abs(poly_area_m) * 1e-6
poly_area_km2

### gc_tarea
NCL's [`gc_tarea`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_tarea.shtml) calculates the area of a triangular patch on the unit sphere

```{admonition} Important Note
Coordinates should be within the valid latitude/longitude range (-90° to 90° and -180° to 180°)
```

#### Grab and Go

##### Area on the unit sphere

NCL's [`gc_tarea`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_tarea.shtml) function finds the area of a triangular patch on the unit sphere, a sphere with radius of 1

In [ ]:
# Unit Sphere radius = 1
from pyproj import Geod

# Latitude and Longitude points for one eighth surface area of a unit sphere
latitudes = [0.0, 0.0, 90.0]  # degrees
longitudes = [0.0, 90.0, 0.0]  # degrees

# Adjust the radius of the spherical datum to describe the unit sphere
radius = 1

geod = Geod(a=radius)
poly_area, _ = geod.polygon_area_perimeter(longitudes, latitudes)
poly_area = abs(poly_area)
poly_area

##### Area on a spherical datum with Earth radius

`pyproj` includes [additional ellipsoid options](https://proj.org/en/stable/usage/ellipsoids.html#built-in-ellipsoid-definitions), but the ellipsoid `sphere` treats the Earth as a sphere with an equal radius of 6370997 meters

In [ ]:
# Normal Sphere: radius = 6370997 m
from pyproj import Geod

# Latitude and Longitude points for one eighth surface area of Earth
latitudes = [0.0, 0.0, 90.0]  # degrees
longitudes = [0.0, 90.0, 0.0]  # degrees

geod = Geod(ellps="sphere")  # radius = 6370997 m
poly_area_m, _ = geod.polygon_area_perimeter(longitudes, latitudes)
poly_area_km2 = abs(poly_area_m) * 1e-6
poly_area_km2

### gc_latlon
NCL's [`gc_latlon`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_latlon.shtml) calculates the  great circle distance (true surface distance) between two points on the globe and interpolates points along the great circle

```{admonition} Important Note
Coordinates should be within the valid latitude/longitude range (-90° to 90° and -180° to 180°)
```

#### Grab and Go

##### Great Circle Distance between Two Points

In [ ]:
from pyproj import Geod

# Latitude and Longitude points
lat1, lon1 = 40.0150, -105.2705  # Boulder
lat2, lon2 = 42.3601, -71.0589  # Boston

geodesic = Geod(ellps="sphere")
_, _, distance_m = geodesic.inv(lon1, lat1, lon2, lat2)

print(f"Distance = {distance_m / 1000} km")

##### Interpolate Points along Great Circle Arc

```{admonition} Important Note
NCL includes the starting and ending points along the great circle arc, while `pyproj` does not
```

In [ ]:
from pyproj import Geod

# Latitude and Longitude points
lat1, lon1 = 40.0150, -105.2705  # Boulder
lat2, lon2 = 42.3601, -71.0589  # Boston

geodesic = Geod(ellps="sphere")
pts = geodesic.npts(lon1, lat1, lon2, lat2, npts=10)

# pts returned in list of longtiude/latitude pairs
for latlon in pts:
    lat, lon = latlon[1], latlon[0]
    print(f"({lat}, {lon})")

### gc_inout
NCL's [`gc_inout`](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_inout.shtml)  determines if a list of latitude/longitude coordinates are inside or outside of spherical polygons

```{admonition} Important Note
`Shapely` works with planar rather than spherical objects and therefore may not work for all cases covered by the corresponding NCL function (e.g. where polygons extend across the poles or antimeridian). <a href="https://spherely.readthedocs.io/en/latest/">`Spherely`</a> is a newer Python package for the manipulation and analysis of objects on the sphere, but is still in early development
```

In [ ]:
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# Point
lat_pt, lon_pt = 40.0150, -105.2705  # Boulder
new_pt = Point(lon_pt, lat_pt)

# List of Points in Latitude/Longitude (North California, Boston, Houston)
lat_pts = [42.3601, 41.4017, 29.5518]
lon_pts = [-71.0589, -124.0417, -95.0982]

# Setup Polygon
lat_lon_coords = tuple(zip(lon_pts, lat_pts))
polygon = Polygon(lat_lon_coords)

# Check if points lie within polygon
polygon.contains(new_pt)

---

## Python Resources
- [pyroj.geod() great circle computations](https://pyproj4.github.io/pyproj/stable/api/geod.html)
- [Astropy Coordinate Systems](https://docs.astropy.org/en/stable/coordinates/representations.html)

## Additional Reading
- [Aviation Formulary for working with great circles](https://www.edwilliams.org/avform147.htm)

## References:

```{footbibliography}
```